In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
users_df = pd.read_csv('takehome_users.csv', encoding='latin-1')
users_df.head()

,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,3,2013-03-19 23:14:52,Bottrill Mitchell,MitchellBottrill@gustr.com,ORG_INVITE,1.363735e+09,0,0,94,1525.0
3,4,2013-05-21 08:09:28,Clausen Nicklas,NicklasSClausen@yahoo.com,GUEST_INVITE,1.369210e+09,0,0,1,5151.0
4,5,2013-01-17 10:14:20,Raw Grace,GraceRaw@yahoo.com,GUEST_INVITE,1.358850e+09,0,0,193,5240.0


In [3]:
users_df.describe()

,object_id,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
count,12000.00000,8.823000e+03,12000.000000,12000.000000,12000.000000,6417.000000
mean,6000.50000,1.379279e+09,0.249500,0.149333,141.884583,5962.957145
std,3464.24595,1.953116e+07,0.432742,0.356432,124.056723,3383.761968
min,1.00000,1.338452e+09,0.000000,0.000000,0.000000,3.000000
25%,3000.75000,1.363195e+09,0.000000,0.000000,29.000000,3058.000000
50%,6000.50000,1.382888e+09,0.000000,0.000000,108.000000,5954.000000
75%,9000.25000,1.398443e+09,0.000000,0.000000,238.250000,8817.000000
max,12000.00000,1.402067e+09,1.000000,1.000000,416.000000,11999.000000


I am going to assume "object_id" is the "user_id"

In [4]:
users_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 10 columns):
object_id                     12000 non-null int64
creation_time                 12000 non-null object
name                          12000 non-null object
email                         12000 non-null object
creation_source               12000 non-null object
last_session_creation_time    8823 non-null float64
opted_in_to_mailing_list      12000 non-null int64
enabled_for_marketing_drip    12000 non-null int64
org_id                        12000 non-null int64
invited_by_user_id            6417 non-null float64
dtypes: float64(2), int64(4), object(4)
memory usage: 937.6+ KB


last_session_creation_time and invited_by_user_id are missing some values. Every user is not going to be a referral, so we can ignore missing values in invited_by_user_id. last_session_creation_time is not going to be useful in predicting the adopted user. So, we can ignore those missing values as well.

In [5]:
users_engagement_df = pd.read_csv('takehome_user_engagement.csv')
users_engagement_df.head()

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1


In [6]:
users_engagement_df.describe()

,user_id,visited
count,207917.000000,207917.0
mean,5913.314197,1.0
std,3394.941674,0.0
min,1.000000,1.0
25%,3087.000000,1.0
50%,5682.000000,1.0
75%,8944.000000,1.0
max,12000.000000,1.0


In [7]:
users_engagement_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 207917 entries, 0 to 207916
Data columns (total 3 columns):
time_stamp    207917 non-null object
user_id       207917 non-null int64
visited       207917 non-null int64
dtypes: int64(2), object(1)
memory usage: 4.8+ MB


In [9]:
users_engagement_df.head(20)

,time_stamp,user_id,visited
0,2014-04-22 03:53:30,1,1
1,2013-11-15 03:45:04,2,1
2,2013-11-29 03:45:04,2,1
3,2013-12-09 03:45:04,2,1
4,2013-12-25 03:45:04,2,1
5,2013-12-31 03:45:04,2,1
6,2014-01-08 03:45:04,2,1
7,2014-02-03 03:45:04,2,1
8,2014-02-08 03:45:04,2,1
9,2014-02-09 03:45:04,2,1


In [10]:
all_data_df = pd.merge(users_engagement_df, users_df, how='left', left_on='user_id', right_on='object_id')
all_data_df.head()

,time_stamp,user_id,visited,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
0,2014-04-22 03:53:30,1,1,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
1,2013-11-15 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2,2013-11-29 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
3,2013-12-09 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
4,2013-12-25 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0


In [11]:
all_data_df.time_stamp = pd.to_datetime(all_data_df.time_stamp)
all_data_df = all_data_df.set_index('time_stamp')
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 207917 entries, 2014-04-22 03:53:30 to 2014-01-26 08:57:12
Data columns (total 12 columns):
user_id                       207917 non-null int64
visited                       207917 non-null int64
object_id                     207917 non-null int64
creation_time                 207917 non-null object
name                          207917 non-null object
email                         207917 non-null object
creation_source               207917 non-null object
last_session_creation_time    207917 non-null float64
opted_in_to_mailing_list      207917 non-null int64
enabled_for_marketing_drip    207917 non-null int64
org_id                        207917 non-null int64
invited_by_user_id            116887 non-null float64
dtypes: float64(2), int64(6), object(4)
memory usage: 20.6+ MB


In [12]:
all_data_df.head()

,user_id,visited,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
time_stamp,,,,,,,,,,,,
2014-04-22 03:53:30,1,1,1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2013-11-15 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2013-11-29 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2013-12-09 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0
2013-12-25 03:45:04,2,1,2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0


In [13]:
all_data_df = all_data_df.sort_index()

In [14]:
all_data_df.head()

,user_id,visited,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
time_stamp,,,,,,,,,,,,
2012-05-31 08:20:06,10012,1,10012,2012-05-31 08:20:06,Spikes Danille,DanilleJSpikes@hotmail.com,ORG_INVITE,1.338452e+09,0,1,225,7104.0
2012-05-31 15:47:36,3428,1,3428,2012-05-31 15:47:36,Spears Arthur,ArthurJSpears@gmail.com,SIGNUP_GOOGLE_AUTH,1.352822e+09,1,0,166,NaN
2012-05-31 17:19:37,9899,1,9899,2012-05-31 17:19:37,Jørgensen Sofie,SofieNJrgensen@yahoo.com,ORG_INVITE,1.338485e+09,0,0,138,9899.0
2012-05-31 21:58:33,1693,1,1693,2012-05-31 21:58:33,Faulkner Hayden,HaydenFaulkner@gmail.com,SIGNUP_GOOGLE_AUTH,1.399932e+09,0,1,50,NaN
2012-06-01 00:17:30,6102,1,6102,2012-06-01 00:17:30,Morrison Natasha,NatashaMorrison@gustr.com,ORG_INVITE,1.339719e+09,0,0,34,6102.0


"adopted user" is a user who has logged into the product on three separate days in at least one sevenday period.

In [15]:
per_user_7day_count = all_data_df.groupby('user_id')['visited'].rolling('7d').count()
per_user_7day_count.head()

user_id  time_stamp         
1        2014-04-22 03:53:30    1.0
2        2013-11-15 03:45:04    1.0
         2013-11-29 03:45:04    1.0
         2013-12-09 03:45:04    1.0
         2013-12-25 03:45:04    1.0
Name: visited, dtype: float64

In [16]:
per_user_7day_count_df = per_user_7day_count.reset_index()
per_user_7day_count_df.head()

,user_id,time_stamp,visited
0,1,2014-04-22 03:53:30,1.0
1,2,2013-11-15 03:45:04,1.0
2,2,2013-11-29 03:45:04,1.0
3,2,2013-12-09 03:45:04,1.0
4,2,2013-12-25 03:45:04,1.0


In [17]:
adopted_users_df = per_user_7day_count_df.loc[per_user_7day_count_df['visited']>=3]
adopted_users_df = adopted_users_df[['user_id']].drop_duplicates()
adopted_users_df = adopted_users_df.reset_index(drop=True)
adopted_users_df.head()

,user_id
0,2
1,10
2,20
3,33
4,42


In [18]:
all_data_df['adopted_user'] = all_data_df['user_id'].apply(lambda x: x in adopted_users_df.values)
all_data_df.head()

,user_id,visited,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
time_stamp,,,,,,,,,,,,,
2012-05-31 08:20:06,10012,1,10012,2012-05-31 08:20:06,Spikes Danille,DanilleJSpikes@hotmail.com,ORG_INVITE,1.338452e+09,0,1,225,7104.0,False
2012-05-31 15:47:36,3428,1,3428,2012-05-31 15:47:36,Spears Arthur,ArthurJSpears@gmail.com,SIGNUP_GOOGLE_AUTH,1.352822e+09,1,0,166,NaN,False
2012-05-31 17:19:37,9899,1,9899,2012-05-31 17:19:37,Jørgensen Sofie,SofieNJrgensen@yahoo.com,ORG_INVITE,1.338485e+09,0,0,138,9899.0,False
2012-05-31 21:58:33,1693,1,1693,2012-05-31 21:58:33,Faulkner Hayden,HaydenFaulkner@gmail.com,SIGNUP_GOOGLE_AUTH,1.399932e+09,0,1,50,NaN,True
2012-06-01 00:17:30,6102,1,6102,2012-06-01 00:17:30,Morrison Natasha,NatashaMorrison@gustr.com,ORG_INVITE,1.339719e+09,0,0,34,6102.0,False


In [27]:
all_data_df['invited_by_user'] = all_data_df['invited_by_user_id'].apply(lambda x: 1 if x>0 else 0)
all_data_df.head()

,user_id,visited,object_id,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user,invited_by_user
time_stamp,,,,,,,,,,,,,,
2012-05-31 08:20:06,10012,1,10012,2012-05-31 08:20:06,Spikes Danille,DanilleJSpikes@hotmail.com,ORG_INVITE,1.338452e+09,0,1,225,7104.0,False,1
2012-05-31 15:47:36,3428,1,3428,2012-05-31 15:47:36,Spears Arthur,ArthurJSpears@gmail.com,SIGNUP_GOOGLE_AUTH,1.352822e+09,1,0,166,NaN,False,0
2012-05-31 17:19:37,9899,1,9899,2012-05-31 17:19:37,Jørgensen Sofie,SofieNJrgensen@yahoo.com,ORG_INVITE,1.338485e+09,0,0,138,9899.0,False,1
2012-05-31 21:58:33,1693,1,1693,2012-05-31 21:58:33,Faulkner Hayden,HaydenFaulkner@gmail.com,SIGNUP_GOOGLE_AUTH,1.399932e+09,0,1,50,NaN,True,0
2012-06-01 00:17:30,6102,1,6102,2012-06-01 00:17:30,Morrison Natasha,NatashaMorrison@gustr.com,ORG_INVITE,1.339719e+09,0,0,34,6102.0,False,1


In [28]:
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 207917 entries, 2012-05-31 08:20:06 to 2014-06-06 14:58:50
Data columns (total 14 columns):
user_id                       207917 non-null int64
visited                       207917 non-null int64
object_id                     207917 non-null int64
creation_time                 207917 non-null object
name                          207917 non-null object
email                         207917 non-null object
creation_source               207917 non-null object
last_session_creation_time    207917 non-null float64
opted_in_to_mailing_list      207917 non-null int64
enabled_for_marketing_drip    207917 non-null int64
org_id                        207917 non-null int64
invited_by_user_id            116887 non-null float64
adopted_user                  207917 non-null bool
invited_by_user               207917 non-null int64
dtypes: bool(1), float64(2), int64(7), object(4)
memory usage: 32.4+ MB


Now, we have the complete data including the target variable "adopted_user". Let's check what independent variable correlate with target variable

Null Hypothesis: There is no relationship between various independent variables tested below and "adopted_user".

Significance Level: 99.9% Confidence (alpha = 0.01)

In [19]:
from scipy import stats

In [25]:
r, pvalue = stats.pearsonr(all_data_df['adopted_user'], all_data_df['opted_in_to_mailing_list'])
print('Correlation: {:0.4f},  p-value: {:0.2e}'.format(r, pvalue))

# Print interpretation:
alpha = 0.01
if pvalue > alpha:
    print('Fail to Reject the Null Hypothesis.')
else:
    print('Reject the Null Hypothesis.')

Correlation: 0.0027,  p-value: 2.14e-01
Fail to Reject the Null Hypothesis.


In [24]:
r, pvalue = stats.pearsonr(all_data_df['adopted_user'], all_data_df['enabled_for_marketing_drip'])
print('Correlation: {:0.4f},  p-value: {:0.2e}'.format(r, pvalue))

# Print interpretation:
alpha = 0.01
if pvalue > alpha:
    print('Fail to Reject the Null Hypothesis.')
else:
    print('Reject the Null Hypothesis.')

Correlation: -0.0005,  p-value: 8.24e-01
Fail to Reject the Null Hypothesis.


In [23]:
r, pvalue = stats.pearsonr(all_data_df['adopted_user'], all_data_df['org_id'])
print('Correlation: {:0.4f},  p-value: {:0.2e}'.format(r, pvalue))

# Print interpretation:
alpha = 0.01
if pvalue > alpha:
    print('Fail to Reject the Null Hypothesis.')
else:
    print('Reject the Null Hypothesis.')

Correlation: 0.0373,  p-value: 5.58e-65
Reject the Null Hypothesis.


In [29]:
r, pvalue = stats.pearsonr(all_data_df['adopted_user'], all_data_df['invited_by_user'])
print('Correlation: {:0.4f},  p-value: {:0.2e}'.format(r, pvalue))

# Print interpretation:
alpha = 0.01
if pvalue > alpha:
    print('Fail to Reject the Null Hypothesis.')
else:
    print('Reject the Null Hypothesis.')

Correlation: 0.0050,  p-value: 2.27e-02
Fail to Reject the Null Hypothesis.


In [30]:
r, pvalue = stats.pearsonr(all_data_df['adopted_user'], all_data_df['last_session_creation_time'])
print('Correlation: {:0.4f},  p-value: {:0.2e}'.format(r, pvalue))

# Print interpretation:
alpha = 0.01
if pvalue > alpha:
    print('Fail to Reject the Null Hypothesis.')
else:
    print('Reject the Null Hypothesis.')

Correlation: 0.6000,  p-value: 0.00e+00
Reject the Null Hypothesis.


Interestingly, referals (invited_by_user) don't make any difference in whether the user will be active or not. "org_id" and "last_session_creation_time" are good indicators to predict whether we have "adopted user" or not